In [ ]:
!pip install langchain
!pip install torch
!pip install sentence_transformers
!pip install faiss-cpu
!pip install huggingface-hub
!pip install pypdf
!pip -q install accelerate
!pip install llama-cpp-python
!pip -q install git+https://github.com/huggingface/transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Us

In [ ]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

from huggingface_hub import hf_hub_download

In [ ]:
#Step 06:Downlaod the Embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.1-GGUF"
model_basename = "mistral-7b-instruct-v0.1.Q3_K_M.gguf"

In [ ]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

mistral-7b-instruct-v0.1.Q3_K_M.gguf:   0%|          | 0.00/3.52G [00:00<?, ?B/s]

In [ ]:
llm = LlamaCpp(
    streaming = True,
    model_path=model_path,
    temperature=0.75,
    n_threads=2, # CPU cores
    n_gpu_layers=32, # Change this value based on your model and your GPU VRAM pool.
    top_p=1,
    verbose=True,
    n_ctx=4096
)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.1-GGUF/snapshots/731a9fc8f06f5f5e2db8a0cf9d256197eb6e05d1/mistral-7b-instruct-v0.1.Q3_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv

# RAG ON PDF


In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader

In [ ]:
#load pdf files
loader = PyPDFDirectoryLoader("/content/sample_data")
data = loader.load()

In [ ]:
print(data)

[Document(page_content="Biography of MS Dhoni For UPSC\nMahendra Singh Dhoni, popularly known as 'Mahi', is one of the most Internationally acclaimed cricket\nplayers and former captain of the Indian Cricket team. A skilled batsman, efficient captain, and flexible\nwicketkeeper, he played dynamic roles within the team that eventually led India to win the Twenty20\nWorld title. A right-handed batsman and wicketkeeper, he is also one of the top finishers in the\nlimited-over format. He delivered incredible performances throughout his career, soaring the legacy of\ncricket in India.\nRead more articles and biographies for UPSC 2023 to ace your preparation.\nEarly Life\n●\nMahendra Singh Dhoni was born in the Ranchi district of Jharkhand on 7 July 1981. He was the\nyoungest of the three children of Pan Singh and Devaki Devi. His family belonged to Hindu\nRajputs and was based in the Langara block of the Almora district in Uttarakhand.\n●\nDuring his schooling at DAV Jawahar Vidya Mandir, h

In [ ]:
#Step 05: Split the Extracted Data into Text Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)

text_chunks = text_splitter.split_documents(data)


In [ ]:
len(text_chunks)

6

In [ ]:
#get the third chunk
text_chunks[2]

Document(page_content="cricket innings with the highest run aggregate of 346. He also received the title ‘Man of the\nSeries' for his incredible performance in the series.BCCI rewarded him with a B-grade contract\nin December 2005.\n●\nDhoni played with his full potential in the series against Pakistan in 2006, scoring 149 runs in\ntwo matches. As a result of his sterling performances throughout the ODI series, Dhoni topped\nthe ICC ODI rankings for batsmen in 2006. He replaced Ricky Ponting and became the fastest\nbatsman to achieve that in just 42 innings. However, he did not deliver a satisfactory\nperformance in the 2006 ICC Champions Trophy and ODI series in South Africa. He indeed\nreceived criticism for his wicket-keeping technique from Syed Kirmani, a former Indian\nwicketkeeper. However, he was listed in the World ODI XI by the International Cricket Council\n(ICC) in 2016 for his remarkable efforts.\n●\nPrior to the 2007 World Cup, Dhoni scored an average of more than 100 in s

In [ ]:
#Step 08: Create Embeddings for each of the Text Chunk
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k": 1}))

In [ ]:
query = "In the 2002- 2003 season for Jharkhand Team"

In [ ]:
qa.run(query)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    3197.75 ms
llama_print_timings:      sample time =      15.66 ms /    29 runs   (    0.54 ms per token,  1852.21 tokens per second)
llama_print_timings: prompt eval time =  343254.38 ms /   744 tokens (  461.36 ms per token,     2.17 tokens per second)
llama_print_timings:        eval time =   15748.56 ms /    28 runs   (  562.45 ms per token,     1.78 tokens per second)
llama_print_timings:       total time =  351658.31 ms /   772 tokens


" Dhoni didn't give a noteworthy performance in the 2002- 2003 season."

# RAG ON LINK

In [ ]:
!pip install unstructured
from langchain.document_loaders import UnstructuredURLLoader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 47.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 50.7 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=12eb78392443f344fabdacaf5b13007e4720adb0bea32f0ec71b1b02632cf3f8
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
urls = [
    'https://www.mosaicml.com/blog/mpt-7b',
    'https://stability.ai/blog/stability-ai-launches-the-first-of-its-stablelm-suite-of-language-models',
    'https://lmsys.org/blog/2023-03-30-vicuna/'
]

In [ ]:
loaders = UnstructuredURLLoader(urls=urls)
data = loaders.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
data

[Document(page_content='Introducing MPT-7B, the first entry in our MosaicML Foundation Series. MPT-7B is a transformer trained from scratch on 1T tokens of text and code. It is open source, available for commercial use, and matches the quality of LLaMA-7B. MPT-7B was trained on the MosaicML platform in 9.5 days with zero human intervention at a cost of ~$200k.\n\nLarge language models (LLMs) are changing the world, but for those outside well-resourced industry labs, it can be extremely difficult to train and deploy these models. This has led to a flurry of activity centered on open-source LLMs, such as the LLaMA series from Meta, the Pythia series from EleutherAI, the StableLM series from StabilityAI, and the OpenLLaMA model from Berkeley AI Research.\n\nToday, we at MosaicML are releasing a new model series called MPT (MosaicML Pretrained Transformer) to address the limitations of the above models and finally provide a commercially-usable, open-source model that matches (and - in many

In [ ]:
# Text Splitter
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(separator='\n',
                                      chunk_size=1000,
                                      chunk_overlap=200)


docs = text_splitter.split_documents(data)

In [ ]:
len(docs)

54

In [ ]:
vectorStore_LINK = FAISS.from_documents(docs, embeddings)

In [ ]:
qa_link = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorStore_LINK.as_retriever(search_kwargs={"k": 1}))

In [ ]:
query = "How good is Vicuna?"

In [ ]:
qa_link.run(query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(

llama_print_timings:        load time =    3339.32 ms
llama_print_timings:      sample time =     151.81 ms /   256 runs   (    0.59 ms per token,  1686.37 tokens per second)
llama_print_timings: prompt eval time =  153969.83 ms /   327 tokens (  470.86 ms per token,     2.12 tokens per second)
llama_print_timings:        eval time =  145528.19 ms /   255 runs   (  570.70 ms per token,     1.75 tokens per second)
llama_print_timings:       total time =  300749.56 ms /   582 tokens


' Based on the preliminary evaluation using GPT-4 as a judge, Vicuna-13B achieves more than 90%* quality of OpenAI ChatGPT and Google Bard while outperforming other models like LLaMA and Stanford Alpaca in more than 90%* of cases.\n\nDonations\nChatbot Arena\nVicuna: An Open-Source Chatbot Impressing GPT-4 with 90%* ChatGPT Quality\nby: The Vicuna Team, Mar 30, 2023\nWe introduce Vicuna-13B, an open-source chatbot trained by fine-tuning LLaMA on user-shared conversations collected from ShareGPT. Preliminary evaluation using GPT-4 as a judge shows Vicuna-13B achieves more than 90%* quality of OpenAI ChatGPT and Google Bard while outperforming other models like LLaMA and Stanford Alpaca in more than 90%* of cases. The cost of training Vicuna-13B is around $300. The code and weights, along with an online demo, are publicly available for non-com'

# RAG ON CSV

In [ ]:
import pandas as pd

df = pd.read_csv('/content/sample_data/HAL.NS.csv')

In [ ]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2023-04-19,1415.025024,1424.250000,1405.699951,1410.550049,1395.069946,794646
1,2023-04-20,1414.500000,1418.675049,1384.275024,1388.074951,1372.841431,1293338
2,2023-04-21,1385.000000,1398.500000,1370.250000,1390.974976,1375.709717,1204638
3,2023-04-24,1397.150024,1417.000000,1385.025024,1412.800049,1397.295288,1286140
4,2023-04-25,1413.500000,1424.500000,1407.550049,1412.224976,1396.726440,921288


In [ ]:
!pip install langchain_experimental
from langchain_experimental.agents import create_csv_agent

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 1.9 MB/s eta 0:00:00


In [ ]:
agent = create_csv_agent(llm,
                         '/content/sample_data/HAL.NS.csv',
                         verbose=True)

In [ ]:
agent

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'df_head': '|    | Date       |    Open |    High |     Low |   Close |   Adj Close |   Volume |\n|---:|:-----------|--------:|--------:|--------:|--------:|------------:|---------:|\n|  0 | 2023-04-19 | 1415.03 | 1424.25 | 1405.7  | 1410.55 |     1395.07 |   794646 |\n|  1 | 2023-04-20 | 1414.5  | 1418.68 | 1384.28 | 1388.07 |     1372.84 |  1293338 |\n|  2 | 2023-04-21 | 1385    | 1398.5  | 1370.25 | 1390.97 |     1375.71 |  1204638 |\n|  3 | 2023-04-24 | 1397.15 | 1417    | 1385.03 | 1412.8  |     1397.3  |  1286140 |\n|  4 | 2023-04-25 | 1413.5  | 1424.5  | 1407.55 | 1412.22 |     1396.73 |   921288 |', 'tools': 'python_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When usin

In [ ]:
agent.run("how many rows are there?")



> Entering new AgentExecutor chain...


Llama.generate: prefix-match hit

llama_print_timings:        load time =    3339.32 ms
llama_print_timings:      sample time =      20.00 ms /    37 runs   (    0.54 ms per token,  1850.28 tokens per second)
llama_print_timings: prompt eval time =  303674.22 ms /   637 tokens (  476.73 ms per token,     2.10 tokens per second)
llama_print_timings:        eval time =   19768.26 ms /    36 runs   (  549.12 ms per token,     1.82 tokens per second)
llama_print_timings:       total time =  323789.43 ms /   673 tokens
Llama.generate: prefix-match hit


Thought: We can get the number of rows using `len(df)`.
Action: python_repl_ast
Action Input: len(df)247


llama_print_timings:        load time =    3339.32 ms
llama_print_timings:      sample time =       6.77 ms /    13 runs   (    0.52 ms per token,  1921.37 tokens per second)
llama_print_timings: prompt eval time =    6346.61 ms /    11 tokens (  576.96 ms per token,     1.73 tokens per second)
llama_print_timings:        eval time =    5971.27 ms /    12 runs   (  497.61 ms per token,     2.01 tokens per second)
llama_print_timings:       total time =   12371.04 ms /    23 tokens


247 rows
Final Answer: 247

> Finished chain.


'247'

In [ ]:
agent.run("Tell Price on 25th April 2023?")



> Entering new AgentExecutor chain...


Llama.generate: prefix-match hit

llama_print_timings:        load time =    3339.32 ms
llama_print_timings:      sample time =      63.52 ms /   120 runs   (    0.53 ms per token,  1889.05 tokens per second)
llama_print_timings: prompt eval time =   22078.96 ms /    18 tokens ( 1226.61 ms per token,     0.82 tokens per second)
llama_print_timings:        eval time =   70400.73 ms /   119 runs   (  591.60 ms per token,     1.69 tokens per second)
llama_print_timings:       total time =   75287.07 ms /   137 tokens
Llama.generate: prefix-match hit


Thought: I should use the `df` dataframe to filter for the date of 25th April 2023 and then find the Close price.
Action: Use the `loc[]` method to filter for the date of 25th April 2023 using the `Date` column, and then use the `iloc[]` method to select the Close price from the resulting dataframe.
Action Input: `df.loc[pd.Timestamp('2023-04-25')]['Close']`Use the `loc[]` method to filter for the date of 25th April 2023 using the `Date` column, and then use the `iloc[]` method to select the Close price from the resulting dataframe. is not a valid tool, try one of [python_repl_ast].


llama_print_timings:        load time =    3339.32 ms
llama_print_timings:      sample time =      41.47 ms /    74 runs   (    0.56 ms per token,  1784.55 tokens per second)
llama_print_timings: prompt eval time =   35318.35 ms /    75 tokens (  470.91 ms per token,     2.12 tokens per second)
llama_print_timings:        eval time =   44520.75 ms /    73 runs   (  609.87 ms per token,     1.64 tokens per second)
llama_print_timings:       total time =   80157.22 ms /   148 tokens


 I should use python_repl_ast to execute the command
Action: python_repl_ast `df.loc[pd.Timestamp('2023-04-25')]['Close']`
Final Answer: The price on 25th April 2023 was 1412.22.

> Finished chain.


'The price on 25th April 2023 was 1412.22.'

# RAG ON YOUTUBE LINK


In [ ]:
!pip install pip install youtube-transcript-api pytube
from langchain.document_loaders import YoutubeLoader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.3 MB/s eta 0:00:00


In [ ]:
loader=YoutubeLoader.from_youtube_url(
    "https://youtu.be/fjHtjT7GO1c?si=YzMspJ5v7DLtdilQ",
    add_video_info=True,
    language=["en","id"],
    translation="en"
)

data=loader.load()

In [ ]:
# Text Splitter
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(separator='\n',
                                      chunk_size=1000,
                                      chunk_overlap=200)


info = text_splitter.split_documents(data)

In [ ]:
info

[Document(page_content="hey I'm Scott from cognition Ai and  today I'm really excited to introduce  you to Devon the first AI software  engineer let me show you an example of  Devon in  action I'm going to ask Devon to  Benchmark the performance of llama and a  couple different API  providers from now on Devon is in the  driver's  seat first Devon makes a step-by-step  plan of how to tackle the  problem after that it builds the whole  project using all the same tools that  human software engineer would use Devon  has its own command  line its own code  editor and even its own  browser in this case Devon decides to  use the browser to pull up API  documentation so that it can read up and  learn how to plug into each of these  apis here Deon runs into an unexpected  error Devon actually decides to add a  debugging print  statement reruns the code with the  debugging print statement and then uses  the error in the logs to figure out how  to fix the  bug finally Devon decides to build and 

In [ ]:
vectorStore_Youtube = FAISS.from_documents(info, embeddings)

In [ ]:
youtube = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorStore_Youtube.as_retriever(search_kwargs={"k": 1}))

In [ ]:
query = "Summarize the Video"

In [ ]:
youtube.run(query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(

llama_print_timings:        load time =    5040.55 ms
llama_print_timings:      sample time =      36.76 ms /    62 runs   (    0.59 ms per token,  1686.75 tokens per second)
llama_print_timings: prompt eval time =  211324.46 ms /   419 tokens (  504.35 ms per token,     1.98 tokens per second)
llama_print_timings:        eval time =   37234.22 ms /    61 runs   (  610.40 ms per token,     1.64 tokens per second)
llama_print_timings:       total time =  248980.57 ms /   480 tokens


' Devon is an AI software engineer that makes plans, builds projects using the same tools human software engineers use, has its own command line, code editor, and browser. It can debug and fix bugs by adding print statements and using error logs. Devon can build and deploy websites with full styling.'